# Amazon Bedrock と Amazon Kendra を用いた Advanced RAG 構築

本ノートブックでは、Amazon Bedrock を生成器、Amazon Kendra を検索器として RAG システムを構築します。下記の順番で、まずは抽象度低く Naive RAG を構築し、動作を理解してから LangChain や LangGraph を用いた Advanced RAG システムの構築を体験します。

1. AWS SDK for Python (boto3) を用いて Naive RAG を構築
2. LangChain を用いて Naive RAG を構築
3. LangGraph を用いて Naive RAG & Advanced RAG を構築

## 1. 事前準備

Ctrl + \` で Code Editor (OSS 版 VS Code) のターミナルを開き、以下のコマンドを実行してください。

```bash:
python3 -m venv .venv
source .venv/bin/activate
pip install -U pip
pip install -r requirements.txt
```

## 2. boto3 を直接用いて RAG を構築

### 必要なモジュールのインポートと boto3 クライアントの準備

AWS SDK for Python (boto3) 等のモジュールをインポートし、事前準備を行います。
ノートブックの各セルを実行するには `Shift + Enter` を押してください。

In [ ]:
import re  # 正規表現
import json
from pprint import pprint

from IPython.display import (
    display,
    Markdown,
)

import boto3
from botocore.exceptions import ClientError

# Amazon Kendra と Amazon Bedrock のクライアント
kendra = boto3.client("kendra", region_name="us-west-2")
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-west-2")

# Amazon Bedrock で利用する Claude 3 Haiku のモデル ID
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

accept = "application/json"
content_type = "application/json"

#### Amazon Kendra のインデックス ID を入力

ワークショップの手順書のタイトルをクリックしてイベントのダッシュボードを開き、パラメーターのリスト (Event Outputs) にある `KendraIndexID` の値を以下にコピーしてください。

In [ ]:
# TODO: 自身の環境に合わせて書き換える
kendra_index_id = "1e74d846-4929-49a4-83bb-00f4c97baf2a"  # 36 文字

### まずは Amazon Bedrock に直接問いかけてみる

`query_text` に好きな質問を入れて Claude 3 Haiku に問いかけてみましょう。

In [ ]:
# ユーザーの質問
query_text = "Amazon Bedrockのモデル評価機能ではどのようなことができますか？"

In [ ]:
# Amazon Bedrock に送るリクエストのボディー
body = json.dumps({
    "max_tokens": 1024,  # 最大出力トークン数
    "messages": [
        {"role": "user", "content": query_text},
        # {"role": "assistant", "content": "いい質問ですね。"},
        # のように user -> assistant -> user -> ... の会話形式にすることも可能
    ],
    # https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html
    "anthropic_version": "bedrock-2023-05-31",  # 固定
    "temperature": 0,  # 出力のランダム度
})

try:
    # Amazon Bedrock の InvokeModel API を実行
    response_bedrock = bedrock_runtime.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=content_type,
    )
    response_body = json.loads(response_bedrock.get("body").read())
    display(Markdown(response_body.get("content")[0]["text"]))
except ClientError as error:
    raise error

#### InvokeModel API の理解を深める

なお、`response_bedrock` の中身は以下のようになっており、`body` キーには `StreamingBody` の形で出力が格納されています。

In [ ]:
response_bedrock

`response_body` は JSON であり、`content` キーの中に辞書のリスト形式で出力文が格納されます。他にも例えば `usage` キーで入力と出力のトークン数を確認でき、実際にどの程度の料金がかかったかを計算することができます。

In [ ]:
pprint(response_body)

### Amazon Kendra で検索する

AWS のインテリジェント検索サービスである Amazon Kendra の検索を試してみましょう。

Kendra でドキュメントを検索するには [Query API](https://docs.aws.amazon.com/ja_jp/kendra/latest/APIReference/API_Query.html) と [Retrieve API](https://docs.aws.amazon.com/ja_jp/kendra/latest/APIReference/API_Retrieve.html) のふたつの方法があります。今回は、RAG との親和性が高い Retrieve API を用いることで、データソースの中のドキュメントから、ユーザーの質問内容に関連する抜粋部分を抽出し、生成 AI への入力としていきます。

なお、Retrieve API では `PageSize` (デフォルト値は10件) を超える抜粋部分が抽出された際、レスポンスがページングされ、追加の `PageSize` 分ごとに結果を再取得していく必要があります。本ワークショップで RAG を実装する際には、最初の10件の抜粋のみを後続の処理に渡していくこととします。

In [ ]:
response_kendra = kendra.retrieve(
    QueryText=query_text,  # 検索クエリ
    IndexId=kendra_index_id,  # Kendra Index ID
    AttributeFilter={
        "EqualsTo": {
            "Key": "_language_code",
            "Value": {
                "StringValue": "ja",  # 日本語ドキュメントを検索 (default: en)
            }
        }
    },
    #PageNumber=1,
    PageSize=3,  # 一度に返すドキュメント抜粋の数 (default: 10)
)

print("Query Text:", query_text)
print("Number of Retrieved Items:", len(response_kendra["ResultItems"]))

for ii, item in enumerate(response_kendra["ResultItems"]):
    print("-" * 30)
    print("Item", ii + 1)
    print("DocumentTitle:", item["DocumentTitle"])
    print("Content")
    pprint(item["Content"])

#### Retrieve API の理解を深める

Retrieve API の `response_kendra` の中にある `ResultItems` キーに、検索で得られたドキュメントの抜粋の情報が格納されています。

In [ ]:
response_kendra

`ResultItems` の各項目の中身は [`RetrieveResultItem`](https://docs.aws.amazon.com/kendra/latest/APIReference/API_RetrieveResultItem.html) オブジェクトです。`RetrieveResulltItem` は以下の情報を含みます。

- `Content`: 検索でヒットしたドキュメントの抜粋部分のテキスト
- `DocumentAttributes`: ドキュメントのソース URI や抜粋のページ番号などの情報
- `DocumentId`: ドキュメントの ID
- `DocumentTitle`: ドキュメントのタイトル
- `DocumentURI`: ドキュメントの URI
- `Id`: ドキュメントの抜粋のユニークな ID
- `ScoreAttribute`: クエリとの関連度合いのスコア (日本語は未対応で `NOT_AVAILABLE` が返る)

In [ ]:
pprint(response_kendra["ResultItems"][0])

以下では、Amazon Kendra の Retrieve API で得られたドキュメントの抜粋を Amazon Bedrock に送るプロンプトに含めることで RAG を実現していきます。

### Amazon Bedrock と Kendra を組み合わせて RAG を実行する

ここまでの情報だけで RAG を実現することができます。LLM に与えるプロンプトの中に Kendra で取得したドキュメントの抜粋を入れ込み、質問に答えてもらうように指示します。

#### RAG 用のプロンプト

サンプルのプロンプトテンプレートは以下の通りです。ここでは、`<excerpt>` (抜粋) タグ内にドキュメントの抜粋を記載し、`<query>` タグ内に質問文を記載しています。タグ名は固定ではなく、任意の値を設定できます。

XML タグを理解するように学習されているのは Claude の特徴であり、XML タグを利用することで構造化されたプロンプトを簡単に構築することができます。詳しくは [Anthropic のプロンプトエンジニアリングガイド](https://docs.anthropic.com/ja/docs/use-xml-tags)を参照してください。

In [ ]:
prompt = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
"""

for item in response_kendra["ResultItems"]:
    prompt += f"<excerpt>{item['Content']}</excerpt>\n"

prompt += f"""</excerpts>

<excerpts>タグ内の情報を参考にして、<query>タグ内のユーザーの質問に答えてください。

<query>
{query_text}
</query>
"""

print(prompt)

#### RAG の実行

上記のプロンプトを利用して Amazon Bedrock の InvokeModel API を実行してみましょう。

In [ ]:
def invoke_model(prompt, model_id="anthropic.claude-3-haiku-20240307-v1:0"):
    body = json.dumps({
        "max_tokens": 1024,
        "messages": [{"role": "user", "content": prompt}],
        "anthropic_version": "bedrock-2023-05-31",
        "temperature": 0,
    })

    try:
        response_bedrock = bedrock_runtime.invoke_model(
            body=body, modelId=model_id, accept=accept, contentType=content_type,
        )
        response_body = json.loads(response_bedrock.get("body").read())
        return response_body.get("content")[0]["text"]
    except ClientError as error:
        raise error

print(invoke_model(prompt, model_id))

#### プロンプトエンジニアリング実践

##### 1. 回答文に XML タグを含めないようにする

`<excerpts>` や `<excerpt>` タグはシステム内部で用いるものであり、ユーザーからするとよくわからない謎の文字列です。
このままではユーザーを不安にしてしまうので、回答生成に条件を追加し、回答に XML タグを含めないようにしましょう。

In [ ]:
prompt = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
"""

for item in response_kendra["ResultItems"]:
    prompt += f"<excerpt>{item['Content']}</excerpt>\n"

prompt += f"""</excerpts>

<excerpts>タグ内の情報を参考にして、<query>タグ内のユーザーの質問に答えてください。

<query>
{query_text}
</query>
"""

rule = """
ただし、<rule>タグ内の回答ルールを遵守してください。

<rule>
- 回答には<excerpts>, <excerpt>, <query>タグを含めないこと。
</rule>
"""

print(prompt + rule)
print("-----")
print(invoke_model(prompt + rule, model_id))

##### 2. 検索インデックスに含まれていない内容は答えないようにする

ここまでのプロンプトでは、Kendra に含まれていない知識に関する質問をされても LLM (Claude 3 Haiku) の持っている知識で答えようとします。

<!--例えば、Amazon SageMaker Studio の上でネイティブに利用できる開発環境は、JupyterLab、RStudio、SageMaker Canvas、Code Editor (OSS 版 Visual Studio Code)、SageMaker Studio Classic なのですが、RAG システムに問い合わせると間違った答え (ハルシネーション) が返ってきます。-->

In [ ]:
query_text = "アマゾン川はどの国を通っていますか？"

# Kendra での検索 - 無関係なドキュメントが検索される
response_kendra = kendra.retrieve(
    QueryText=query_text,  # 検索クエリ
    IndexId=kendra_index_id,  # Kendra Index ID
    AttributeFilter={
        "EqualsTo": {
            "Key": "_language_code",
            "Value": {
                "StringValue": "ja",  # 日本語ドキュメントを検索 (default: en)
            }
        }
    },
    #PageNumber=1,
    PageSize=3,  # 一度に返すドキュメント抜粋の数 (default: 10)
)

# LLM へのプロンプト
prompt = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
"""

for item in response_kendra["ResultItems"]:
    prompt += f"<excerpt>{item['Content']}</excerpt>\n"  # 質問と無関係なドキュメントを追加

prompt += f"""</excerpts>

<excerpts>タグ内の情報を参考にして、<query>タグ内のユーザーの質問に答えてください。

<query>
{query_text}
</query>
"""

rule = """
ただし、<rule>タグ内の回答ルールを遵守してください。

<rule>
- 回答には<excerpts>, <excerpt>, <query>タグを含めないこと。
</rule>
"""

print(prompt + rule)
print("-----")

# RAG 実行
print(invoke_model(prompt + rule, model_id))

今回は、ナレッジベース (Kendra) から関連するドキュメントが検索できなかった場合には無理に回答しないようにプロンプトを工夫してみましょう。

In [ ]:
rule = """
ただし、<rule>タグ内の回答ルールを遵守してください。

<rule>
- 回答には<excerpts>, <excerpt>, <query>タグを含めないこと。
- 回答は<answer>タグで囲むこと。
- 回答する前に、質問へ回答するために必要な情報が与えられているか考え、<thinking>タグに判断の理由を書いてください。
</rule>
"""

print(prompt + rule)
print("-----")

# RAG 実行
output = invoke_model(prompt + rule, model_id)
print(output)
print("-----")

match = re.search(r"<answer>(.*?)</answer>", output, re.DOTALL)
if match:
    answer = match.group(1).strip()
    print(answer)


この、最終的な答えを提供する前に Claude に考える時間を与える手法は「思考の連鎖 (Chain of Thoughts; CoT)」プロンプトと呼ばれます。
Claude の段階的な推論と最終的な応答を区別しやすくするために、`<thinking>` や `<answer>` タグを使うことが推奨されています。
詳しくは [Anthropic のプロンプトエンジニアリングガイド](https://docs.anthropic.com/ja/docs/let-claude-think#claude-2)を参照してください。

##### 3. 回答の根拠となる引用を付ける

本セクションの締めくくりとして、回答文だけでなく、回答の根拠となる文書を引用するようにしてみましょう。

まず、プロンプトに Kendra で検索したドキュメントの抜粋だけでなく、ドキュメント ID やタイトルも含めるようにします。

In [ ]:
query_text = "Amazon Bedrockって何ですか？"

response_kendra = kendra.retrieve(
    QueryText=query_text,  # 検索クエリ
    IndexId=kendra_index_id,  # Kendra Index ID
    AttributeFilter={
        "EqualsTo": {
            "Key": "_language_code",
            "Value": {
                "StringValue": "ja",  # 日本語ドキュメントを検索 (default: en)
            }
        }
    },
    #PageNumber=1,
    PageSize=10,  # 一度に返すドキュメント抜粋の数 (default: 10)
)

prompt = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
"""

for item in response_kendra["ResultItems"]:
    prompt += "<excerpt>"
    prompt += f"<document_id>{item['DocumentId']}</document_id>"
    prompt += f"<title>{item['DocumentTitle']}</title>"
    prompt += f"<content>{item['Content']}</content>"
    prompt += "</excerpt>\n"


prompt += f"""</excerpts>

<excerpts>タグ内の情報を参考にして、<query>タグ内のユーザーの質問に答えてください。

<query>
{query_text}
</query>
"""

print(prompt)

追加したドキュメント ID やタイトルの情報をもとに、回答の根拠となるドキュメントを引用するようにします。ここでは、LLM の出力を XML タグで構造化し、それをレンダリングする際にパースする方針を取ります。

In [ ]:
rule = """
回答内で検索結果からの情報を参照する場合は、情報が見つかったソースのドキュメントへの引用を含める必要があります。
各結果には、参照すべき対応するdocument_idとtitleを付与します。
回答に複数のドキュメントの抜粋からの情報が含まれている場合、<sources>には複数の<source>が含まれる可能性があることに注意してください。
<excerpts>を回答で直接引用しないでください。あなたの仕事は、ユーザーの質問に可能な限り簡潔に答えることです。
以下の形式で回答を出力する必要があります。フォーマットとスペースに注意して、正確に従ってください。
<answer>
  <answer_part>
    <text>
      最初の回答テキスト
    </text>
    <sources>
      <source>
        <document_id>document_id</document_id>
        <title>title</title>
      </source>
    </sources>
  </answer_part>
  <answer_part>
    <text>
      2番目の回答テキスト
    </text>
    <sources>
      <source>
        <document_id>document_id</document_id>
        <title>title</title>
      </source>
    </sources>
  </answer_part>
</answer>
"""

In [ ]:
print(prompt + rule)
print("-----")

# RAG 実行
output = invoke_model(prompt + rule, model_id)
print(output)
print("-----")

正規表現を使って XML タグをパースします

In [ ]:
def parse_answer(xml):
    # XMLデータから<answer_part>タグで囲まれた部分を抽出する
    answer_parts = re.findall(r'<answer_part>(.*?)</answer_part>', xml, re.DOTALL)
    result = ""  # 結果を格納する変数
    source_refs = ""  # ソース情報を格納する変数
    source_index = 1  # ソースに付与する連番
    unique_refs = set()  # 重複するソースのタイトルを避けるための集合
    source_dic = {}  # ソースのタイトルと番号のマッピング
    for part in answer_parts:
        # 回答の本文を抽出する
        text = re.search(r'<text>(.*?)</text>', part, re.DOTALL).group(1).strip()
        # ソース情報を抽出する
        sources = re.findall(r'<source>(.*?)</source>', part, re.DOTALL)
        source_str = ""  # 現在の部分のソース情報 (e.g. [1]) を格納する文字列
        for source in sources:
            document_id = re.search(r'<document_id>(.*?)</document_id>', source).group(1)
            title = re.search(r'<title>(.*?)</title>', source).group(1)
            # ソースのタイトルが重複する場合は、前に出現したものと同じ番号を付与する
            if title in unique_refs:
                source_str += f" \[{source_dic[title]}\]"
            else:
                source_str += f" \[{source_index}\]"
                unique_refs.add(title)
                source_dic[title] = source_index
                source_refs += f"\[{source_index}\] [{title}]({document_id})  \n"  # ソース情報を追加
                source_index += 1
        # 回答の本文とソース情報を結果に追加する
        result += f"{text}{source_str}\n\n"
    # 最後にソースの連番とタイトルを追加する
    return result + "\n\n" + source_refs


display(Markdown(parse_answer(output)))

### Pre-retrieval: クエリ拡張

クエリ拡張は、単一のクエリを複数のクエリに拡張することで多様な検索結果を取得し、生成される回答の適合性を高めるための手法です。クエリ拡張にも、単純に複数の異なるクエリを作成するマルチクエリ (multi query) や、元の質問を分解して個々の質問に答えるためのクエリを作成するサブクエリ (sub query) などさまざまなアプローチがありますが、今回はシンプルなマルチクエリのアプローチを採用しています。LLM は generator と同様に Claude 3 Haiku を用いてクエリを拡張します。

必ずしもユーザーのクエリとソースドキュメントの表現が一致しているわけではありません。「ナレッジベース」と「Knowledge Bases」等、日本語と英語との表記揺れや、類義語、タイポなど、さまざまな要因で初期のクエリでは回答に必要な情報が十分に取得できないことがあります。クエリを拡張し、類似するキーワードを複数用いることで、キーワード検索とベクトル検索のハイブリッド検索のような、確実性と曖昧性を兼ね備えた検索が実現されることが期待されます。

In [ ]:
n_queries = 3

prompt = f"""
検索エンジンに入力するクエリを最適化し、様々な角度から検索を行うことで、より適切で幅広い検索結果が得られるようにします。 
以下の<question>タグ内にはユーザーの入力した質問文が入ります。
この質問文に基づいて、{n_queries}個の検索用クエリを生成してください。
各クエリは30トークン以内とし、日本語と英語を適切に混ぜて使用することで、広範囲の文書が取得できるようにしてください。

<question>
{query_text}
<question>
"""

print(invoke_model(prompt))

このままだとプログラム的に扱いづらいので出力フォーマットを指定します。

In [ ]:
output_format = ""
for i in range(n_queries):
    output_format += f"query {i+1}: fill a query here\n"

format = f"""
生成されたクエリは、<format>タグ内のフォーマットに従って出力してください。

<format>
{output_format}
</format>
"""

queries = invoke_model(prompt + output_format)
print(queries)

あとは正規表現で抜き出してあげれば後段の Kendra への入力にすることができます。

In [ ]:
queries = [query for query in queries.split("\n") if re.match(r'query \d+:[^:]+$', query)]
queries

### Post-retrieval: 検索結果の関連度評価

このステップでは検索結果が、元のユーザーからの質問に関連したものになっているかを評価します。検索で得られたドキュメントの抜粋 (チャンク) というのは必ずしも質問に回答するための情報を含んでいるとは限らず、誤った回答を誘発するような内容も含んでしまっていることがあります。例えば、“[Corrective Retrieval Augmented Generation](https://arxiv.org/abs/2401.15884)” \[Shi-Qi Yan et al. (2024)\] の論文で提案されている手法 (Corrective RAG; CRAG) では、検索結果の関連度を評価し、その評価結果をもとに最終回答を生成します。CRAG ではドキュメントの抜粋をそれぞれ Correct (正確) / Ambiguous (曖昧) / Incorrect (不正確) のカテゴリに分類しますが、今回は簡単のために関連しているか否かの yes / no の二値に分類します。LLM は他のステップと同様に Claude 3 Haiku を用います。

In [ ]:
query_text = "Amazon Bedrock で Claude 3 Sonnet の基盤モデルに関する情報を取得する Python コードを教えて"
response_kendra = kendra.retrieve(
    QueryText=query_text,  # 検索クエリ
    IndexId=kendra_index_id,  # Kendra Index ID
    AttributeFilter={"EqualsTo": {"Key": "_language_code", "Value": {"StringValue": "ja",}}},
    PageSize=5,  # 一度に返すドキュメント抜粋の数 (default: 10)
)

result_items = []

for ii, item in enumerate(response_kendra["ResultItems"]):
    result_item = ""
    result_item += "-" * 30 + "\n"
    result_item += f"Item {ii+1}\n"
    result_item += f"DocumentTitle:{item['DocumentTitle']}\n"
    result_item += "Content\n"
    result_item += item["Content"] + "\n"
    result_items.append(result_item)

pprint(result_items)

In [ ]:
for result_item in result_items:
    prompt = f"""
    あなたは、ユーザーからの質問と検索で得られたドキュメントの関連性を評価する専門家です。
    <excerpt>タグ内は、検索により取得したドキュメントの抜粋です。

    <excerpt>{result_item}</excerpt>
    
    <question>タグ内は、ユーザーからの質問です。

    <question>{query_text}</question>
    
    このドキュメントの抜粋は、ユーザーの質問に回答するための正確な情報を含んでいるかを慎重に判断してください。
    正確な情報を含んでいる場合は 'yes'、含んでいない場合は 'no' のバイナリスコアを返してください。
    バイナリスコアは<related></related>タグ内に格納してください。
    """

    print(invoke_model(prompt))
    print("-----")

ここでは、Kendra の Retrieve API で得られたドキュメントの抜粋ひとつひとつに対して関連度評価を行っています。クエリ拡張と組み合わせた場合 (元のクエリ + 3つの拡張クエリ) は、4クエリ×10抜粋=40抜粋分、Claude 3 Haiku を呼び出すことになります。1抜粋当たり、プロンプト全体で800トークン前後になるため、一回の検索における関連度評価だけで30,000トークン程度消費するケースがあることになります。レイテンシーの影響も最小限に抑えるため、クエリ拡張と同様に非同期処理として実行するのが望ましいですが、運用の際には RPM (Requests processed per minute) や TPM (Tokens processed per minute) のクォータには注意してください。

## Naive RAG を LangChain で構築する

LangChain は、大規模言語モデル (LLM) を活用したアプリケーション開発のためのオープンソースフレームワークです。様々な言語モデル、データソース、API を組み合わせて、コンテキストを理解し推論できるアプリケーションを構築できます。プロンプトテンプレート、メモリ管理、モジュール化されたアーキテクチャなどの機能を備えており、チャットボット、質問応答システム、要約ツールなど、多様なユースケースに適用可能です。

langchain_aws は、LangChain と AWS を連携させる Python パッケージです。Amazon Bedrock、Amazon Kendra などのサービスと連携でき、AmazonKendraRetriever や ChatBedrock など AWS 固有のコンポーネントを提供しています。AmazonKendraRetriever は Amazon Kendra を利用したドキュメント検索、ChatBedrock は Amazon Bedrock の LLM を使ったチャットモデルです。

langchain_aws を使えば、AWS の豊富な機能と LangChain の柔軟性を組み合わせ、実用的で拡張性の高い自然言語処理アプリケーションを構築できます。LLM の能力をさらに引き出し、外部データと統合させた高度なアプリケーション開発が可能になります。

### 必要なモジュールのインポート

In [ ]:
from langchain_aws import AmazonKendraRetriever
from langchain_aws import ChatBedrock
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
import langchain

langchain.verbose = True

### AmazonKendraRetriever

In [ ]:
retriever = AmazonKendraRetriever(
    index_id=kendra_index_id,
    region_name="us-west-2",
    attribute_filter={"EqualsTo": {"Key": "_language_code", "Value": {"StringValue": "ja"}}},
    top_k=10,
)
retriever.invoke(input=query_text)

### ChatBedrock

In [ ]:

chat = ChatBedrock(
    model_id=model_id,
    region_name="us-west-2",
    model_kwargs={
        "temperature": 0,
        "max_tokens": 1024,
    }
)
chat.invoke(input=query_text)

### ChatPromptTemplate

In [ ]:
system_template = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
{context}
</excerpts>

<excerpts>タグ内の情報を参考にして、ユーザーの質問に答えてください。
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("human", "{input}")
    ]
)

### Chain

ここまでは boto3 を直接叩くのと対して変わらないですが、LamgChain はその名の通り各コンポーネントをチェーンのように繋ぐことができるのが便利なところです。単純な RAG システムを構築するためのモジュールが用意されており、以下のようにして全体のフローをひとつのチェーンとして実行できます。

In [ ]:

question_answer_chain = create_stuff_documents_chain(chat, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

chain.invoke({"input": query_text})


#### 参考
LangChain において文書を要約したり質問に答えたりする際に、複数の文書をどのように処理するかはいくつか方式があります。主に以下の4つのタイプがあります。
1. stuff
- 全ての文書を1つのプロンプトに詰め込んで (stuff)、1回の API 呼び出しで処理する最もシンプルな方式。
- 文書が小さく数が少ない場合に適している。
- 大きな文書にはあまり向かない。
2. map_reduce
- 各文書に対して個別に LLM を適用し (map)、その結果を新しい文書とみなす。
- それらの新しい文書を別の文書結合チェーンに渡して単一の出力を得る (reduce)。
- 必要に応じて map された文書を再帰的に圧縮する。
- 大きな文書を扱うのに適している。
- API 呼び出し回数が多くなる。
3. refine
- 入力文書をループして反復的に回答を更新していく方式。
- 各文書について、現在の中間回答と共に LLM に渡し、新しい回答を得る。
- 1度に1つの文書しか LLM に渡さないため、多くの文書を分析するのに適している。
- stuff よりもはるかに多くの LLM 呼び出しが必要。
4. map_rerank
- 各文書に対してプロンプトを実行し、タスクを完了させるだけでなく、回答の確からしさもスコア付けさせる。
- スコアが最も高い回答を返す。
以上のように、扱う文書の量や目的に応じて適切な chain type を選択することで、LangChain を使った文書の要約や質問応答をより効果的に行うことができます。

https://github.com/langchain-ai/langchain/tree/master/libs/langchain/langchain/chains/combine_documents

## LangGraph

LangGraph は、LangChain をベースに構築された大規模言語モデルを用いたステートフルなマルチエージェントアプリケーションを開発するための Python ライブラリです。主な特徴は以下の通りです。

1. サイクル(循環)の実装が容易で、LLM をループで呼び出しながら次のアクションを決定するようなフローを簡単に実装できる。
2. グラフベースでエージェントのワークフローを定義し、グラフの状態を変更することで動的な応答が可能。
3. アプリケーション全体の状態を管理する GraphState を定義でき、状態管理が容易。
4. 複数の独立したエージェントを連携させて協調動作させることができる。
5. 永続化機能により、長時間実行のマルチセッションアプリケーションに対応。
6. LangChain エコシステムと統合されており、様々な機能を活用できる。

他にも、Advanced RAG のような複雑なフローをグラフベースのアーキテクチャによって簡単に実装できることもメリットです。LangGraph を用いると高度な自然言語処理アプリケーションをモジュール形式で容易に実装できます。

例えば、以下のようなフローを作る際、Retrieve や Grade などのノードをエッジ (矢印) で繋ぐことで状態 (state) 遷移の流れを構築します。
条件分岐も conditional edge と呼ばれるコンポーネントを利用することで簡単に実現できます。
![langgraph_crag](img/crag.png)
出典: [Corrective RAG (CRAG) - langchain-ai/langgraph](https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb)

<!--![langgraph_crag](img/crag.png)
出典: [Corrective RAG (CRAG) - langchain-ai/langgraph](https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag.ipynb?ref=blog.langchain.dev)-->

### 各モジュールの構築

今回構築するグラフ構造は下図の通りです。質問が投げられたら `entry_point` を経由して、ひとつ目の条件分岐に入ります。`n_queries` パラメータが1以上であれば `generate_queries` ノードで `n_queries` 個のクエリ拡張を行い、`retrieve` ノードで検索を行います。ふたつ目の条件分岐では、`grade_documents_enabled` パラメータが `"Yes"` であれば `grade_documents` ノードを実行し、フィルタされたドキュメントの抜粋を元に `generate` ノードで回答生成を行います。

![graph](img/final_graph.png)

#### State

ノード間の状態 (state) は辞書形式で持つことにします。

In [ ]:
from typing import Dict, TypedDict
from langchain_core.output_parsers import StrOutputParser


class GraphState(TypedDict):
    """Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]


#### Entry Point

はじめに条件分岐を置きたいため、グラフのエントリーポイントとしてはパススルーのノードをセットします。

In [ ]:
def entry_point(state):
    """Pass through"""
    return state

#### ひとつ目の条件分岐

`n_queries` パラメータが1以上であれば `generate_queries_enable`、0以下であれば `generate_queries_not_enable` と判断するエッジのロジックを書きます。

![decide_to_generate_queries](img/decide_to_generate_queries.png)

In [ ]:
def decide_to_generate_queries(state):
    """
    Determines whether to generate queries, or use the original query.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---DECIDE TO GENERATE QUERIES---")
    n_queries = state["keys"]["n_queries"]

    if n_queries > 0:
        print(
            "---DECISION: GENERATE QUERIES---"
        )
        return "generate_queries_enabled"
    else:
        print("---DECISION: NOT GENERATE QUERIES---")
        return "generate_queries_not_enabled"

#### Generate Queries: クエリ拡張

元のクエリ (`question`) に加えて `n_queries` のクエリを拡張するノードを定義します。

![generate_queries](img/generate_queries.png)

In [ ]:
def generate_queries(state):
    """Generate a variety of queries (RAG-Fusion).

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): The updated graph state with generated queries.
    """
    print("---GENERATE QUERIES---")
    state_dict = state["keys"]
    question = state_dict["question"]
    n_queries = state_dict["n_queries"]

    llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        region_name="us-west-2",
        model_kwargs={
            "temperature": 0,
            "max_tokens": 512,
        }
    )

    output_format = ""
    for i in range(n_queries):
        output_format += f"{i+1}: fill a query here\n"

    prompt_template = """
検索エンジンに入力するクエリを最適化し、様々な角度から検索を行うことで、より適切で幅広い検索結果が得られるようにします。 
具体的には、類義語や日本語と英語の表記揺れを考慮し、多角的な視点からクエリを生成します。

以下の<question>タグ内にはユーザーの入力した質問文が入ります。
この質問文に基づいて、{n_queries}個の検索用クエリを生成してください。
各クエリは30トークン以内とし、日本語と英語を適切に混ぜて使用することで、広範囲の文書が取得できるようにしてください。

生成されたクエリは、<format>タグ内のフォーマットに従って出力してください。

<example>
question: Knowledge Bases for Amazon Bedrock ではどのベクトルデータベースを使えますか？
query 1: Knowledge Bases for Amazon Bedrock vector databases engine DB store
query 2: Amazon Bedrock ナレッジベース ベクトル ベクター エンジン データベース ストア インデックス
query 3: Amazon Bedrock RAG 検索拡張生成 埋め込み embedding ベクトル データベース エンジン
</example>

<format>
{output_format}
</format> 

<question>
{question}
</question>
""".format(
        output_format=output_format,
        n_queries=n_queries,
        question="{question}",  # as-is
    )
    prompt = ChatPromptTemplate.from_template(prompt_template)

    chain = prompt | llm | StrOutputParser() | (lambda x: x.split("\n"))
    queries = chain.invoke({"question": question})
    queries = [query.replace('"', '') for query in queries]
    queries = [query for query in queries if re.match(r'^\d+:[^:]+$', query)]  # 数字:文字列の項目のみ抽出
    queries = [f"0: {question}"] + queries
    print(queries)

    state_dict["queries"] = queries

    return {"keys": state_dict}

#### Retrieve: Amazon Kendra を用いた検索

Amazon Kendra でクエリに基づいて検索するノードを定義します。

![retrieve](img/retrieve.png)

In [ ]:
def retrieve(state):
    """Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    retriever = AmazonKendraRetriever(
        index_id=kendra_index_id,
        region_name="us-west-2",
        attribute_filter={"EqualsTo": {"Key": "_language_code", "Value": {"StringValue": "ja"}}},
        top_k=10
    )
    documents = []
    if "queries" in state_dict:
        queries = state_dict["queries"]
        for query in queries:
            documents.extend(retriever.invoke(query))
    else:
        documents = retriever.invoke(question)
    state_dict["documents"] = documents
    return {"keys": state_dict}

#### ふたつ目の条件分岐

`grade_documents_enabled` パラメータが `"Yes"` であれば `grade_documents_enabled` を返し、`"No"` であれば `grade_documents_not_enabled` を返すロジックを書きます。

![decide_to_grade_documents](img/decide_to_grade_documents.png)

In [ ]:
def decide_to_grade_documents(state):
    """
    Determines whether to grade documents or not.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---DECIDE TO GRADE DOCUMENTS---")
    grade_documents_enabled = state["keys"]["grade_documents_enabled"]

    if grade_documents_enabled == "Yes":
        print(
            "---DECISION: GRADE DOCUMENTS---"
        )
        return "grade_documents_enabled"
    else:
        print("---DECISION: NOT GRADE DOCUMENTS---")
        return "grade_documents_not_enabled"

#### Grade Documents: 検索結果の関連度評価

Amazon Kendra の検索結果が元の質問 (`question`) に関連したものになっているかを判断し、関連しないようであれば除外するノードを定義する。

![grade_documents](img/grade_documents.png)

In [ ]:
def grade_documents(state):
    """Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with relevant documents
    """

    print("---CHECK RELEVANCE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    llm = ChatBedrock(
        model_id=model_id,
        region_name="us-west-2",
        model_kwargs={
            "temperature": 0,
            "max_tokens": 128,
        }
    )

    system_template = """
あなたはユーザーからの質問と取得したドキュメントの関連性を評価します。

<excerpt>タグ内は、検索により取得したドキュメントの抜粋です。

<excerpt>
{context}
</excerpt>

ドキュメントの抜粋が、ユーザーの質問に関連するものであるかを判別してください。
関連していれば 'yes'、していなければ 'no' の binary score を返してください。
binary score は<related>タグに格納し、出力は <related>yes</related> もしくは <related>no</related> のみとしてください。
"""

    #parser = XMLOutputParser(tags=["related"])
    #print(parser.get_format_instructions())

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_template),
            ("human", "{question}")
        ]
    )
    

    chain = prompt | llm | StrOutputParser()

    filtered_docs = []

    for doc in documents:
        output = chain.invoke(
            {
                "question": question,
                "context": doc.page_content,
                #"format_instructions": parser.get_format_instructions(),
            }
        )
        pattern = r"<related>(.*?)</related>"
        match = re.search(pattern, output)

        if match:
            score = match.group(1)
            if score == "yes":
                print("---GRADE: DOCUMENT RELEVANT---")
                filtered_docs.append(doc)
            else:
                print("---GRADE: DOCUMENT NOT RELEVANT---")
                continue
        else:
            print("マッチする文字列がありません。")

    search = "Yes" if len(filtered_docs) == 0 else "No"

    state_dict["documents"] = filtered_docs
    state_dict["run_web_search"] = search

    return {
        "keys": state_dict
    }

#### Generate

検索で得られたドキュメントの抜粋をもとに質問への回答生成を行うノードを定義する。

![generate](img/generate.png)

In [ ]:
def generate(state):
    """Generate documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    system_template = """
あなたは親切なチャットアシスタントです。

<excerpts>タグにはユーザーが知りたい情報を検索した結果のドキュメントの抜粋が含まれています。
ドキュメントの抜粋は複数あり、それぞれの抜粋が<excerpt>タグで囲まれています。

<excerpts>
{context}
</excerpts>

<excerpts>タグ内の情報を参考にして、ユーザーの質問に答えてください。
ただし、<rule>タグ内の回答ルールを遵守してください。

<rule>
- 回答には<excerpts>, <excerpt>, <query>タグを含めないこと。
- 回答は<answer>タグで囲むこと。
- 回答する前に、質問へ回答するために必要な情報が与えられているか考え、<thinking>タグに判断の理由を書いてください。
</rule>
"""

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_template),
            ("human", "{question}")
        ]
    )

    llm = ChatBedrock(
        model_id=model_id,
        region_name="us-west-2",
        model_kwargs={
            "temperature": 0,
            "max_tokens": 1024,
        }
    )

    rag_chain = prompt | llm | StrOutputParser()

    output = rag_chain.invoke({"context": documents, "question": question})

    match = re.search(r"<answer>(.*?)</answer>", output, re.DOTALL)
    if match:
        generation = match.group(1).strip()
    else:
        generation = ""

    state_dict["generation"] = generation
    return {
        "keys": state_dict
    }
    

### Graph 全体を構築

![final_graph](img/final_graph.png)

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# ノードを追加
workflow.add_node("entry_point", entry_point)
workflow.add_node("generate_queries", generate_queries)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)

# グラフのフローを定義
workflow.set_entry_point("entry_point")
workflow.add_conditional_edges(
    "entry_point",
    decide_to_generate_queries,
    {
        "generate_queries_enabled": "generate_queries",
        "generate_queries_not_enabled": "retrieve",
    },
)
workflow.add_edge("generate_queries", "retrieve")
workflow.add_conditional_edges(
    "retrieve",
    decide_to_grade_documents,
    {
        "grade_documents_enabled": "grade_documents",
        "grade_documents_not_enabled": "generate",
    }
)
workflow.add_edge("grade_documents", "generate")
workflow.add_edge("generate", END)

# グラフのコンパイル
app = workflow.compile()

### Graph を実行

LangGraph で構築したグラフは `app.stream(inputs)` で実行できます。for loop を用いることでグラフ構造に従ってそれぞれのノードが実行されていきます。

In [ ]:
def invoke_graph(app, inputs):
    for output in app.stream(inputs):
        for key, value in output.items():
            pprint(f"Node '{key}':")
        print("\n---\n")

    pprint(value["keys"]["generation"])

以下で `question` や `n_queries`、`grade_documents_enabled` の値を変えることで Advanced RAG の手法をオンオフしながら質問してみましょう。

<b> ＊ 今回の実装では負荷の軽減のため Kendra の検索や関連度評価は同期的に実行していますが、非同期的に並列で実行することでレイテンシーを大幅に削減することができます。 </b>

In [ ]:

# 質問集
question = "Knowledge Bases for Amazon Bedrock ではどういったベクトルデータベースを利用できますか？"
#question = "Amazon Kendraを使ってWebサイトのコンテンツを検索可能にしたいと考えています。クロールの対象とするURLを制限する方法はありますか?"
#question = "Kendra で使用できるデータソースを全部教えて"
#question = "Amazon Kendra がサポートしているユーザーアクセス制御の方法は？"
#question = "Amazon Kendra の検索分析のメトリクスには何がありますか？"
#question = "Amazon Bedrock で Claude 3 Sonnet の基盤モデルに関する情報を取得する Python コードを教えて"
#question = "ナレッジベースでのembeddingモデルの選択肢は？"
#question = "Amazon Kendraで検索結果のランキングロジックをカスタマイズできますか？"
#question = "Amazon Kendraにはどんなエディションがありますか？"
#question = "Amazon Bedrock でモデルにアクセスするには何が必要ですか？"
#question = "Bedrockのagent機能は東京リージョンでは使えますか？"
#question = "外国語学習を効果的に進めるコツとは？"
#question = "Amazon CloudFrontの仕組みについて説明してください。"
#question = "Amazon EC2の特徴は何ですか？"

inputs = {"keys": {
    "question": question, 
    "n_queries": -1,  # generate_queries_not_enabled
    "grade_documents_enabled": "No",
    #"n_queries": 3,  # generate_queries_enabled
    #"grade_documents_enabled": "Yes",
}}

invoke_graph(app, inputs)

## まとめ

本ノートブックでは、Amazon Bedrock と Amazon Kendra を用いて Naive RAG から Advanced RAG までを段階的に構築する方法について学びました。

今回の Naive RAG の構成は、Amazon Kendra でドキュメントを検索し、その検索結果を Amazon Bedrock の Claude モデルに渡して回答を生成するシンプルなアプローチです。
boto3 を直接使う方法と、LangChain を使ってコンポーネント化する方法を紹介しました。

Advanced RAGでは、いくつかの手法を組み合わせることで、より高度な検索と回答生成を実現しました。

- Pre-retrieval のクエリ拡張により、元のクエリだけでなく関連する複数のクエリで検索することで、より多様で適切な検索結果を得られるようにしました。
- Post-retrieval の関連度評価では、検索結果の各ドキュメントが元の質問に関連しているかを判定し、無関係なドキュメントを除外することで、回答の精度向上を図りました。

最後に、LangGraph というグラフベースのフレームワークを使って、Advanced RAG の一連の処理フローをモジュール化し、パラメータを切り替えるだけで簡単に実行できるようにしました。

本ノートブックで紹介したテクニックを応用することで、LLM とナレッジベース、検索エンジンを組み合わせた高度な質問応答システムを構築することができます。実務への適用の際は、非同期処理の導入などパフォーマンスチューニングも必要になるでしょう。

LLM の性能は目覚ましい発展を遂げていますが、すべての知識を言語モデル内に詰め込むのは現実的ではありません。社内の情報資産を安全に利活用しながら、言語モデルの汎用的な能力と組み合わせる本アプローチは、企業の生産性を高め、イノベーションを加速する有望な手段になると考えられます。ぜひ参考にしていただければ幸いです。